<a href="https://colab.research.google.com/github/heebonpark/-_-/blob/main/%EB%B6%80%EC%8B%A4_%EC%9C%A0%EC%A7%80%EC%8B%9C%EC%84%A4_%EC%A1%B0%EA%B1%B4%EB%A7%A4%ED%95%91_%EB%8B%A4%EC%8B%9C0915.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from google.colab import auth
from google.auth import default
import gspread
from google.colab import drive

# --- 0. Google Drive 마운트 ---
try:
  drive.mount('/content/drive')
  print("✅ Google Drive 마운트 성공.")
except Exception as e:
  print(f"❌ Google Drive 마운트 오류: {e}")

# --- 0-2. Google 인증 ---
try:
  auth.authenticate_user()
  creds, _ = default()
  gc = gspread.authorize(creds)
  print("✅ Google 인증 성공.")
except Exception as e:
  print(f"❌ Google 인증 오류: {e}")

# --- 1. CSV 파일 로드 ---
csv_file_path = '/content/drive/MyDrive/시설/유지/8월마감 조건추출 결합_선택컬럼_0831.csv'
try:
  df_csv = pd.read_csv(csv_file_path, low_memory=False)
  print("\n📂 CSV 미리보기:")
  display(df_csv.head())
  print("\n📂 CSV 컬럼:")
  display(df_csv.columns)
except FileNotFoundError:
  print(f"❌ CSV 파일을 찾을 수 없습니다: {csv_file_path}")
  df_csv = None
except Exception as e:
  print(f"❌ CSV 파일 로드 오류: {e}")
  df_csv = None

# --- 1-2. Google Sheet 로드 ---
google_sheet_url = 'https://docs.google.com/spreadsheets/d/1IGWbBlPYqS7mvv9gfFISyvg6vLeJSAHXfyewt_W5pqE/edit?usp=sharing'
sheet_name_read = '부실'
df_sheet = None

if gc:
    try:
        sh = gc.open_by_url(google_sheet_url)
        try:
            worksheet_read = sh.worksheet(sheet_name_read)
        except gspread.WorksheetNotFound:
            print(f"❌ 시트 '{sheet_name_read}'를 찾을 수 없습니다.")
            print("✅ 사용 가능한 시트 목록:", [ws.title for ws in sh.worksheets()])
            worksheet_read = None

        if worksheet_read:
            data = worksheet_read.get_all_values()
            if data:
                df_sheet = pd.DataFrame(data[1:], columns=data[0])
                print("\n📄 Google Sheet 미리보기:")
                display(df_sheet.head())
                print("\n📄 Google Sheet 컬럼:")
                display(df_sheet.columns)
            else:
                print(f"❌ '{sheet_name_read}' 시트에 데이터가 없습니다.")
                df_sheet = None

    except gspread.SpreadsheetNotFound:
        print(f"❌ 해당 URL에서 Google Sheet를 찾을 수 없습니다: {google_sheet_url}")
        df_sheet = None
    except Exception as e:
        print(f"❌ Google Sheet 로드 중 오류 발생: {e}")
        df_sheet = None

# --- 2. 병합 및 업데이트 ---
if df_csv is not None and df_sheet is not None:
    try:
        # 계약번호 문자열 변환
        df_sheet['계약번호'] = df_sheet['계약번호'].astype(str)
        df_csv['계약번호'] = df_csv['계약번호'].astype(str)

        # 필요한 컬럼만 추출
        csv_cols_to_merge = ['계약번호', '실적채널', '영업구역정보', '서비스(소)', 'KTT월정료(조정)', '설치주소','사업용구분']
        df_csv_subset = df_csv[csv_cols_to_merge]

        # 병합
        df_merged = pd.merge(df_sheet, df_csv_subset, on='계약번호', how='left', suffixes=('', '_csv'))

        # 텍스트 필드 병합
        for col in ['실적채널', '영업구역정보', '서비스(소)', '설치주소','사업용구분']:
            if f'{col}_csv' in df_merged.columns:
                if col in df_merged.columns:
                    df_merged[col] = df_merged[f'{col}_csv'].combine_first(df_merged[col])
                df_merged = df_merged.drop(columns=[f'{col}_csv'])

        # 월정료 컬럼 생성 또는 업데이트
        if '월정료(VAT미포함)' not in df_merged.columns:
            print("🔔 '월정료(VAT미포함)' 컬럼이 없어 새로 생성합니다.")
            df_merged['월정료(VAT미포함)'] = None

        if 'KTT월정료(조정)' in df_merged.columns:
            df_merged['월정료(VAT미포함)'] = df_merged['KTT월정료(조정)'].combine_first(df_merged['월정료(VAT미포함)'])
            df_merged = df_merged.drop(columns=['KTT월정료(조정)'])

        # ✅ 필터링: 계약상태_중에서 '청구설변', '청구취소' 제외
        if '계약상태_중' in df_merged.columns:
            before_count = len(df_merged)
            df_merged = df_merged[~df_merged['계약상태_중'].isin(['청구설변', '청약취소'])]
            after_count = len(df_merged)
            print(f"\n🧹 '계약상태_중' 필터 적용: {before_count - after_count}건 제외됨")

        # 컬럼 순서 정렬
        new_order_cols = [col for col in ['실적채널', '영업구역정보', '서비스(소)', '월정료(VAT미포함)', '설치주소','사업용구분'] if col in df_merged.columns]
        existing_cols = [col for col in df_merged.columns if col not in new_order_cols]
        df_sheet_updated = df_merged[new_order_cols + existing_cols]

        print("\n✅ 병합된 DataFrame:")
        display(df_sheet_updated.head())
        print("\n📌 컬럼 순서 확인:")
        display(df_sheet_updated.columns)

        # --- 3. Google Sheet에 다시 쓰기 ---
        sheet_name_write = '부실0915_업데이트'  # 원본 보호용

        try:
            try:
                worksheet_write = sh.worksheet(sheet_name_write)
                print(f"📝 '{sheet_name_write}' 시트에 기존 데이터가 있어 clear 후 업데이트합니다.")
                worksheet_write.clear()
            except gspread.WorksheetNotFound:
                print(f"📄 '{sheet_name_write}' 시트를 새로 만듭니다.")
                worksheet_write = sh.add_worksheet(title=sheet_name_write, rows="1000", cols="30")

            # NaN, inf 처리 (Google Sheet 저장 오류 방지)
            df_sheet_updated = df_sheet_updated.replace([float('inf'), float('-inf')], None)
            df_sheet_updated = df_sheet_updated.where(pd.notnull(df_sheet_updated), None)

            # 리스트 변환 후 저장
            data_to_write = [df_sheet_updated.columns.values.tolist()] + df_sheet_updated.values.tolist()
            worksheet_write.update(data_to_write)
            print(f"\n✅ Google Sheet '{sheet_name_write}'에 데이터 저장 완료.")

        except Exception as e:
            print(f"❌ Google Sheet 저장 중 오류: {e}")

    except Exception as e:
        print(f"❌ 병합 및 업데이트 중 오류 발생: {e}")

else:
    print("\n⚠️ CSV 또는 Google Sheet 불러오기에 실패하여 병합 작업이 중단되었습니다.")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Google Drive 마운트 성공.
✅ Google 인증 성공.

📂 CSV 미리보기:


,관리본부명,관리지사명,고객번호,고객명,계약번호,서비스번호,서비스(대),서비스(중),서비스(소),상호,...,요금구분,정지희망종료일,계약최초서비스게시일.1,영업구역정보,제외사유,매출구분,실적채널,고알프,정지기간,정지유형
0,충남/충북본부,충남서부지사,30784870,이상범,51586788,62190953,기본서비스,텔레캅아이,GiGAeyes i-slim,이상범,...,대상,NaN,2.022040e+13,Q000503,NaN,NaN,SP,N,NaN,NaN
1,강남/서부본부,강동지사,30782144,김향순,52181422,63372130,기본서비스,텔레캅아이,GiGAeyes i-slim,김향순,...,대상,NaN,2.022040e+13,NaN,NaN,NaN,SP,N,NaN,NaN
2,강북/강원본부,고양지사,30783713,이광재,51584522,62185954,기본서비스,텔레캅아이,GiGAeyes i-pass,주식회사 에스엔소프트,...,대상,NaN,2.022040e+13,G000301,NaN,NaN,SP,N,NaN,NaN
3,충남/충북본부,천안지사,30783261,장혜정,51583723,62184004,기본서비스,텔레캅아이,GiGAeyes i-view,장혜정,...,대상,NaN,2.022040e+13,Q000205,NaN,NaN,SP,N,NaN,NaN
4,부산/경남본부,남부산지사,30784637,이범자,51906386,62874108,기본서비스,텔레캅아이,GiGAeyes i-slim,이범자,...,대상,NaN,2.022040e+13,I000101,NaN,NaN,SP,N,NaN,NaN



📂 CSV 컬럼:


Index(['관리본부명', '관리지사명', '고객번호', '고객명', '계약번호', '서비스번호', '서비스(대)', '서비스(중)',
       '서비스(소)', '상호', '고객구분', '사업용구분', '주민등록번호', '사업자번호', '계약상태(대)', '설치주소',
       'KTT월정료(조정)', '계약시작일', '계약종료일', '영업자명', '정지시작일자', '종목', '업태', '업종_대',
       '업종_중', '결합구분', '결합상품명', '결합약정개월수', '결합시작일', '타사전환', 'E-MAIL', '휴대폰',
       '계약최초서비스게시일', '시설구분', '요금구분', '정지희망종료일', '계약최초서비스게시일.1', '영업구역정보',
       '제외사유', '매출구분', '실적채널', '고알프', '정지기간', '정지유형'],
      dtype='object')


📄 Google Sheet 미리보기:


...


🧹 '계약상태_중' 필터 적용: 508건 제외됨

✅ 병합된 DataFrame:


,실적채널,영업구역정보,서비스(소),월정료(VAT미포함),설치주소,사업용구분,순번,계약번호,청구번호,고객번호,...,계약상태_대,계약상태_중,우편번호,주소,최초서비스개시일,서비스개시일시,서비스종료일시,관리본부,관리지사,계약일자
0,SP,J000605,GiGAeyes Basic,55000.0,서울 구로구 구로동 235 한신아이티타워 1214호,비사업용,1,51394537,41508200,30679589,...,정지,일시정지,152-768,서울 구로구 구로동 235 한신아이티타워 1214호,2021-05-07,2024-10-02 0:00,2025-10-01 23:59,서부본부,관악지사,2021-05-03
1,SP,M000305,GiGAeyes i-slim,17000.0,경북 안동시 와룡면 산야리 산야리 1266번지,비사업용,2,51396853,41510661,30680448,...,정지,일시정지,760-915,경북 안동시 와룡면 산야리 산야리 1266번지,2021-05-07,2025-05-07 0:00,2026-05-06 23:59,대구/경북본부,구미지사,2023-12-18
2,SP,M000305,GiGAeyes i-slim,17000.0,경북 영주시 영주동 365-43번지 와플대학 영주캠퍼스,비사업용,3,51400484,41514513,30681771,...,정지,일시정지,750-010,경북 영주시 영주동 365-43번지 와플대학 영주캠퍼스,2021-05-12,2025-05-12 0:00,2026-05-11 23:59,대구/경북본부,구미지사,2021-05-10
3,SP,O000702,GiGAeyes i-slim,27000.0,전남 목포시 상동 893-2번지 2층,비사업용,4,51401196,41712326,30784303,...,정지,일시정지,530-826,전남 목포시 상동 893-2번지 2층,2021-05-12,2025-05-12 0:00,2026-05-11 23:59,전남/전북본부,목포지사,2021-05-11
4,SP,M000305,GiGAeyes i-slim,47000.0,경북 영양군 영양읍 상원리 상원리 302-1번지,비사업용,5,51402415,41516527,30683482,...,정지,일시정지,764-803,경북 영양군 영양읍 상원리 상원리 302-1번지,2021-05-12,2025-05-12 0:00,2026-05-11 23:59,대구/경북본부,구미지사,2021-05-12



📌 컬럼 순서 확인:


Index(['실적채널', '영업구역정보', '서비스(소)', '월정료(VAT미포함)', '설치주소', '사업용구분', '순번',
       '계약번호', '청구번호', '고객번호', '조회구분', '(정지/설변)기간', '이벤트시작일', '당해년도 이벤트횟수',
       '고객명', '상호', '법인명', '대표자명', '활동내역', '활동내역등록일', '활동내역등록자', '부실가입자 사유',
       '정지시작일', '정지만료일', '외국인여부', '주민등록번호', '사업자등록번호', '법인번호', '계약상태_대',
       '계약상태_중', '우편번호', '주소', '최초서비스개시일', '서비스개시일시', '서비스종료일시', '관리본부',
       '관리지사', '계약일자'],
      dtype='object')

📄 '부실0915_업데이트' 시트를 새로 만듭니다.

✅ Google Sheet '부실0915_업데이트'에 데이터 저장 완료.


In [ ]:
# 📌 필요한 패키지 설치 (최초 1회만 실행)
!pip install --upgrade gspread gspread_dataframe

import pandas as pd
import gspread
from google.colab import auth
from google.auth import default

# === 1. 구글 인증 ===
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

# === 2. 구글 시트 열기 ===
sheet_url = "https://docs.google.com/spreadsheets/d/1IGWbBlPYqS7mvv9gfFISyvg6vLeJSAHXfyewt_W5pqE/edit?usp=sharing"
spreadsheet = gc.open_by_url(sheet_url)

# 시트 객체
ws_bsil = spreadsheet.worksheet("부실")
ws_jipo = spreadsheet.worksheet("지표")

# === 3. 데이터 불러오기 ===
df_bsil = pd.DataFrame(ws_bsil.get_all_records())
df_jipo = pd.DataFrame(ws_jipo.get_all_values())

# 컬럼명 정리
df_bsil.columns = df_bsil.columns.str.strip()

# === 4. 필요한 컬럼만 선택 ===
use_cols = ["계약번호", "본부", "지사", "L형/i형", "월정료(VAT미포함)"]
df = df_bsil[use_cols].copy()
df["월정료"] = pd.to_numeric(df["월정료(VAT미포함)"], errors="coerce")

# === 5. 집계 ===
agg = df.groupby(["본부","지사","L형/i형"]).agg(
    건수=("계약번호", "count"),
    월정료=("월정료", "sum")
).reset_index()

pivot = agg.pivot_table(
    index=["본부","지사"],
    columns="L형/i형",
    values=["건수","월정료"],
    fill_value=0
)
pivot.columns = [f"{val}_{col}" for val, col in pivot.columns]
pivot = pivot.reset_index()

# === 6. 지사/본부 리스트 ===
jisa_raw = df_jipo.iloc[16:66,1].tolist()
jisa_list = [x for x in jisa_raw if x not in ["", "소계", "합계"]]
bonbu_list = ["강남/서부", "강북/강원", "부산/경남", "전남/전북", "충남/충북", "대구/경북"]

# === 7. 보조 함수 ===
def fmt(val):
    return f"{int(val):,}" if pd.notnull(val) else "0"

# === 8. 본부 집계 (I5:N10) + 합계 (I11:N11) ===
total_bonbu = [0,0,0,0,0,0]
for i, bonbu in enumerate(bonbu_list, start=5):
    sub = pivot[pivot["본부"]==bonbu]
    row = [
        int(sub["건수_L형"].sum()),
        int(sub["건수_i형"].sum()),
        int(sub["건수_L형"].sum() + sub["건수_i형"].sum()),
        int(sub["월정료_L형"].sum()/1000),
        int(sub["월정료_i형"].sum()/1000),
        int((sub["월정료_L형"].sum() + sub["월정료_i형"].sum())/1000)
    ]
    ws_jipo.update(f"I{i}:N{i}", [[fmt(x) for x in row]])
    total_bonbu = [a+b for a,b in zip(total_bonbu, row)]
ws_jipo.update("I11:N11", [[fmt(x) for x in total_bonbu]])

# === 9. 지사 집계 (I17:N65) + 소계 + 전체합계 (I66:N66) ===
row_idx = 17
total_all = [0,0,0,0,0,0]
for bonbu in bonbu_list:
    sub_bonbu = pivot[pivot["본부"]==bonbu]
    for jisa in sub_bonbu["지사"].unique():
        if jisa in jisa_list:
            sub = sub_bonbu[sub_bonbu["지사"]==jisa]
            row = [
                int(sub["건수_L형"].sum()),
                int(sub["건수_i형"].sum()),
                int(sub["건수_L형"].sum() + sub["건수_i형"].sum()),
                int(sub["월정료_L형"].sum()/1000),
                int(sub["월정료_i형"].sum()/1000),
                int((sub["월정료_L형"].sum() + sub["월정료_i형"].sum())/1000)
            ]
            ws_jipo.update(f"I{row_idx}:N{row_idx}", [[fmt(x) for x in row]])
            total_all = [a+b for a,b in zip(total_all, row)]
            row_idx += 1
    # 소계
    row = [
        int(sub_bonbu["건수_L형"].sum()),
        int(sub_bonbu["건수_i형"].sum()),
        int(sub_bonbu["건수_L형"].sum() + sub_bonbu["건수_i형"].sum()),
        int(sub_bonbu["월정료_L형"].sum()/1000),
        int(sub_bonbu["월정료_i형"].sum()/1000),
        int((sub_bonbu["월정료_L형"].sum() + sub_bonbu["월정료_i형"].sum())/1000)
    ]
    ws_jipo.update(f"I{row_idx}:N{row_idx}", [[fmt(x) for x in row]])
    total_all = [a+b for a,b in zip(total_all, row)]
    row_idx += 1
# 전체 합계
ws_jipo.update("I66:N66", [[fmt(x) for x in total_all]])

print("✅ 지표 시트 업데이트 완료 (본부/지사별 + 소계 + 합계)")

# Task
매핑된 '출동/영상'과 'L형/i형' 열을 포함하여 "부실0915_업데이트" Google Sheet를 업데이트합니다.

## Bm 시트 로드

### Subtask:
Google Sheet에서 BM 시트를 로드합니다.


**Reasoning**:
Load the 'BM' sheet from the Google Sheet and display the head of the resulting DataFrame.



In [ ]:
sheet_name_bm = 'BM'
df_bm = None

if gc:
    try:
        sh = gc.open_by_url(google_sheet_url)
        try:
            worksheet_bm = sh.worksheet(sheet_name_bm)
            print(f"\n📄 Google Sheet '{sheet_name_bm}' 로드 중...")
        except gspread.WorksheetNotFound:
            print(f"❌ 시트 '{sheet_name_bm}'를 찾을 수 없습니다.")
            print("✅ 사용 가능한 시트 목록:", [ws.title for ws in sh.worksheets()])
            worksheet_bm = None

        if worksheet_bm:
            data_bm = worksheet_bm.get_all_values()
            if data_bm:
                df_bm = pd.DataFrame(data_bm[1:], columns=data_bm[0])
                print(f"\n📄 Google Sheet '{sheet_name_bm}' 미리보기:")
                display(df_bm.head())
                print(f"\n📄 Google Sheet '{sheet_name_bm}' 컬럼:")
                display(df_bm.columns)
            else:
                print(f"❌ '{sheet_name_bm}' 시트에 데이터가 없습니다.")
                df_bm = None

    except gspread.SpreadsheetNotFound:
        print(f"❌ 해당 URL에서 Google Sheet를 찾을 수 없습니다: {google_sheet_url}")
        df_bm = None
    except Exception as e:
        print(f"❌ Google Sheet 로드 중 오류 발생: {e}")
        df_bm = None
else:
    print("\n⚠️ Google 인증에 실패하여 Google Sheet 로드 작업이 중단되었습니다.")

NameError: name 'gc' is not defined

**Reasoning**:
The variable `gc` was not defined in the current session. Re-execute the code block that initializes `gc` and loads the initial dataframes to ensure all necessary variables are available.



In [ ]:
import pandas as pd
from google.colab import auth
from google.auth import default
import gspread
from google.colab import drive

# --- 0. Google Drive 마운트 ---
try:
  drive.mount('/content/drive')
  print("✅ Google Drive 마운트 성공.")
except Exception as e:
  print(f"❌ Google Drive 마운트 오류: {e}")

# --- 0-2. Google 인증 ---
try:
  auth.authenticate_user()
  creds, _ = default()
  gc = gspread.authorize(creds)
  print("✅ Google 인증 성공.")
except Exception as e:
  print(f"❌ Google 인증 오류: {e}")

# --- 1. CSV 파일 로드 ---
csv_file_path = '/content/drive/MyDrive/시설/유지/8월마감 조건추출 결합_선택컬럼_0831.csv'
try:
  df_csv = pd.read_csv(csv_file_path, low_memory=False)
  print("\n📂 CSV 미리보기:")
  display(df_csv.head())
  print("\n📂 CSV 컬럼:")
  display(df_csv.columns)
except FileNotFoundError:
  print(f"❌ CSV 파일을 찾을 수 없습니다: {csv_file_path}")
  df_csv = None
except Exception as e:
  print(f"❌ CSV 파일 로드 오류: {e}")
  df_csv = None

# --- 1-2. Google Sheet 로드 ---
google_sheet_url = 'https://docs.google.com/spreadsheets/d/1IGWbBlPYqS7mvv9gfFISyvg6vLeJSAHXfyewt_W5pqE/edit?usp=sharing'
sheet_name_read = '부실'
df_sheet = None

if gc:
    try:
        sh = gc.open_by_url(google_sheet_url)
        try:
            worksheet_read = sh.worksheet(sheet_name_read)
        except gspread.WorksheetNotFound:
            print(f"❌ 시트 '{sheet_name_read}'를 찾을 수 없습니다.")
            print("✅ 사용 가능한 시트 목록:", [ws.title for ws in sh.worksheets()])
            worksheet_read = None

        if worksheet_read:
            data = worksheet_read.get_all_values()
            if data:
                df_sheet = pd.DataFrame(data[1:], columns=data[0])
                print("\n📄 Google Sheet 미리보기:")
                display(df_sheet.head())
                print("\n📄 Google Sheet 컬럼:")
                display(df_sheet.columns)
            else:
                print(f"❌ '{sheet_name_read}' 시트에 데이터가 없습니다.")
                df_sheet = None

    except gspread.SpreadsheetNotFound:
        print(f"❌ 해당 URL에서 Google Sheet를 찾을 수 없습니다: {google_sheet_url}")
        df_sheet = None
    except Exception as e:
        print(f"❌ Google Sheet 로드 중 오류 발생: {e}")
        df_sheet = None

# --- 2. 병합 및 업데이트 ---
if df_csv is not None and df_sheet is not None:
    try:
        # 계약번호 문자열 변환
        df_sheet['계약번호'] = df_sheet['계약번호'].astype(str)
        df_csv['계약번호'] = df_csv['계약번호'].astype(str)

        # 필요한 컬럼만 추출
        csv_cols_to_merge = ['계약번호', '실적채널', '영업구역정보', '서비스(소)', 'KTT월정료(조정)', '설치주소','사업용구분']
        df_csv_subset = df_csv[csv_cols_to_merge]

        # 병합
        df_merged = pd.merge(df_sheet, df_csv_subset, on='계약번호', how='left', suffixes=('', '_csv'))

        # 텍스트 필드 병합
        for col in ['실적채널', '영업구역정보', '서비스(소)', '설치주소','사업용구분']:
            if f'{col}_csv' in df_merged.columns:
                if col in df_merged.columns:
                    df_merged[col] = df_merged[f'{col}_csv'].combine_first(df_merged[col])
                df_merged = df_merged.drop(columns=[f'{col}_csv'])

        # 월정료 컬럼 생성 또는 업데이트
        if '월정료(VAT미포함)' not in df_merged.columns:
            print("🔔 '월정료(VAT미포함)' 컬럼이 없어 새로 생성합니다.")
            df_merged['월정료(VAT미포함)'] = None

        if 'KTT월정료(조정)' in df_merged.columns:
            df_merged['월정료(VAT미포함)'] = df_merged['KTT월정료(조정)'].combine_first(df_merged['월정료(VAT미포함)'])
            df_merged = df_merged.drop(columns=['KTT월정료(조정)'])

        # ✅ 필터링: 계약상태_중에서 '청구설변', '청구취소' 제외
        if '계약상태_중' in df_merged.columns:
            before_count = len(df_merged)
            df_merged = df_merged[~df_merged['계약상태_중'].isin(['청구설변', '청약취소'])]
            after_count = len(df_merged)
            print(f"\n🧹 '계약상태_중' 필터 적용: {before_count - after_count}건 제외됨")

        # 컬럼 순서 정렬
        new_order_cols = [col for col in ['실적채널', '영업구역정보', '서비스(소)', '월정료(VAT미포함)', '설치주소','사업용구분'] if col in df_merged.columns]
        existing_cols = [col for col in df_merged.columns if col not in new_order_cols]
        df_sheet_updated = df_merged[new_order_cols + existing_cols]

        print("\n✅ 병합된 DataFrame:")
        display(df_sheet_updated.head())
        print("\n📌 컬럼 순서 확인:")
        display(df_sheet_updated.columns)

        # --- 3. Google Sheet에 다시 쓰기 ---
        sheet_name_write = '부실0915_업데이트'  # 원본 보호용

        try:
            try:
                worksheet_write = sh.worksheet(sheet_name_write)
                print(f"📝 '{sheet_name_write}' 시트에 기존 데이터가 있어 clear 후 업데이트합니다.")
                worksheet_write.clear()
            except gspread.WorksheetNotFound:
                print(f"📄 '{sheet_name_write}' 시트를 새로 만듭니다.")
                worksheet_write = sh.add_worksheet(title=sheet_name_write, rows="1000", cols="30")

            # NaN, inf 처리 (Google Sheet 저장 오류 방지)
            df_sheet_updated = df_sheet_updated.replace([float('inf'), float('-inf')], None)
            df_sheet_updated = df_sheet_updated.where(pd.notnull(df_sheet_updated), None)

            # 리스트 변환 후 저장
            data_to_write = [df_sheet_updated.columns.values.tolist()] + df_sheet_updated.values.tolist()
            worksheet_write.update(data_to_write)
            print(f"\n✅ Google Sheet '{sheet_name_write}'에 데이터 저장 완료.")

        except Exception as e:
            print(f"❌ Google Sheet 저장 중 오류: {e}")

    except Exception as e:
        print(f"❌ 병합 및 업데이트 중 오류 발생: {e}")

else:
    print("\n⚠️ CSV 또는 Google Sheet 불러오기에 실패하여 병합 작업이 중단되었습니다.")

Mounted at /content/drive
✅ Google Drive 마운트 성공.
✅ Google 인증 성공.

📂 CSV 미리보기:


,관리본부명,관리지사명,고객번호,고객명,계약번호,서비스번호,서비스(대),서비스(중),서비스(소),상호,...,요금구분,정지희망종료일,계약최초서비스게시일.1,영업구역정보,제외사유,매출구분,실적채널,고알프,정지기간,정지유형
0,충남/충북본부,충남서부지사,30784870,이상범,51586788,62190953,기본서비스,텔레캅아이,GiGAeyes i-slim,이상범,...,대상,NaN,2.022040e+13,Q000503,NaN,NaN,SP,N,NaN,NaN
1,강남/서부본부,강동지사,30782144,김향순,52181422,63372130,기본서비스,텔레캅아이,GiGAeyes i-slim,김향순,...,대상,NaN,2.022040e+13,NaN,NaN,NaN,SP,N,NaN,NaN
2,강북/강원본부,고양지사,30783713,이광재,51584522,62185954,기본서비스,텔레캅아이,GiGAeyes i-pass,주식회사 에스엔소프트,...,대상,NaN,2.022040e+13,G000301,NaN,NaN,SP,N,NaN,NaN
3,충남/충북본부,천안지사,30783261,장혜정,51583723,62184004,기본서비스,텔레캅아이,GiGAeyes i-view,장혜정,...,대상,NaN,2.022040e+13,Q000205,NaN,NaN,SP,N,NaN,NaN
4,부산/경남본부,남부산지사,30784637,이범자,51906386,62874108,기본서비스,텔레캅아이,GiGAeyes i-slim,이범자,...,대상,NaN,2.022040e+13,I000101,NaN,NaN,SP,N,NaN,NaN



📂 CSV 컬럼:


Index(['관리본부명', '관리지사명', '고객번호', '고객명', '계약번호', '서비스번호', '서비스(대)', '서비스(중)',
       '서비스(소)', '상호', '고객구분', '사업용구분', '주민등록번호', '사업자번호', '계약상태(대)', '설치주소',
       'KTT월정료(조정)', '계약시작일', '계약종료일', '영업자명', '정지시작일자', '종목', '업태', '업종_대',
       '업종_중', '결합구분', '결합상품명', '결합약정개월수', '결합시작일', '타사전환', 'E-MAIL', '휴대폰',
       '계약최초서비스게시일', '시설구분', '요금구분', '정지희망종료일', '계약최초서비스게시일.1', '영업구역정보',
       '제외사유', '매출구분', '실적채널', '고알프', '정지기간', '정지유형'],
      dtype='object')


📄 Google Sheet 미리보기:


...


🧹 '계약상태_중' 필터 적용: 508건 제외됨

✅ 병합된 DataFrame:


,실적채널,영업구역정보,서비스(소),월정료(VAT미포함),설치주소,사업용구분,순번,계약번호,청구번호,고객번호,...,계약상태_대,계약상태_중,우편번호,주소,최초서비스개시일,서비스개시일시,서비스종료일시,관리본부,관리지사,계약일자
0,SP,J000605,GiGAeyes Basic,55000.0,서울 구로구 구로동 235 한신아이티타워 1214호,비사업용,1,51394537,41508200,30679589,...,정지,일시정지,152-768,서울 구로구 구로동 235 한신아이티타워 1214호,2021-05-07,2024-10-02 0:00,2025-10-01 23:59,서부본부,관악지사,2021-05-03
1,SP,M000305,GiGAeyes i-slim,17000.0,경북 안동시 와룡면 산야리 산야리 1266번지,비사업용,2,51396853,41510661,30680448,...,정지,일시정지,760-915,경북 안동시 와룡면 산야리 산야리 1266번지,2021-05-07,2025-05-07 0:00,2026-05-06 23:59,대구/경북본부,구미지사,2023-12-18
2,SP,M000305,GiGAeyes i-slim,17000.0,경북 영주시 영주동 365-43번지 와플대학 영주캠퍼스,비사업용,3,51400484,41514513,30681771,...,정지,일시정지,750-010,경북 영주시 영주동 365-43번지 와플대학 영주캠퍼스,2021-05-12,2025-05-12 0:00,2026-05-11 23:59,대구/경북본부,구미지사,2021-05-10
3,SP,O000702,GiGAeyes i-slim,27000.0,전남 목포시 상동 893-2번지 2층,비사업용,4,51401196,41712326,30784303,...,정지,일시정지,530-826,전남 목포시 상동 893-2번지 2층,2021-05-12,2025-05-12 0:00,2026-05-11 23:59,전남/전북본부,목포지사,2021-05-11
4,SP,M000305,GiGAeyes i-slim,47000.0,경북 영양군 영양읍 상원리 상원리 302-1번지,비사업용,5,51402415,41516527,30683482,...,정지,일시정지,764-803,경북 영양군 영양읍 상원리 상원리 302-1번지,2021-05-12,2025-05-12 0:00,2026-05-11 23:59,대구/경북본부,구미지사,2021-05-12



📌 컬럼 순서 확인:


Index(['실적채널', '영업구역정보', '서비스(소)', '월정료(VAT미포함)', '설치주소', '사업용구분', '순번',
       '계약번호', '청구번호', '고객번호', '조회구분', '(정지/설변)기간', '이벤트시작일', '당해년도 이벤트횟수',
       '고객명', '상호', '법인명', '대표자명', '활동내역', '활동내역등록일', '활동내역등록자', '부실가입자 사유',
       '정지시작일', '정지만료일', '외국인여부', '주민등록번호', '사업자등록번호', '법인번호', '계약상태_대',
       '계약상태_중', '우편번호', '주소', '최초서비스개시일', '서비스개시일시', '서비스종료일시', '관리본부',
       '관리지사', '계약일자'],
      dtype='object')

📝 '부실0915_업데이트' 시트에 기존 데이터가 있어 clear 후 업데이트합니다.

✅ Google Sheet '부실0915_업데이트'에 데이터 저장 완료.


**Reasoning**:
Now that the `gc` object is properly initialized, attempt to load the 'BM' sheet again as per the subtask instructions.



In [ ]:
sheet_name_bm = 'BM'
df_bm = None

if gc:
    try:
        sh = gc.open_by_url(google_sheet_url)
        try:
            worksheet_bm = sh.worksheet(sheet_name_bm)
            print(f"\n📄 Google Sheet '{sheet_name_bm}' 로드 중...")
        except gspread.WorksheetNotFound:
            print(f"❌ 시트 '{sheet_name_bm}'를 찾을 수 없습니다.")
            print("✅ 사용 가능한 시트 목록:", [ws.title for ws in sh.worksheets()])
            worksheet_bm = None

        if worksheet_bm:
            data_bm = worksheet_bm.get_all_values()
            if data_bm:
                df_bm = pd.DataFrame(data_bm[1:], columns=data_bm[0])
                print(f"\n📄 Google Sheet '{sheet_name_bm}' 미리보기:")
                display(df_bm.head())
                print(f"\n📄 Google Sheet '{sheet_bm}' 컬럼:")
                display(df_bm.columns)
            else:
                print(f"❌ '{sheet_name_bm}' 시트에 데이터가 없습니다.")
                df_bm = None

    except gspread.SpreadsheetNotFound:
        print(f"❌ 해당 URL에서 Google Sheet를 찾을 수 없습니다: {google_sheet_url}")
        df_bm = None
    except Exception as e:
        print(f"❌ Google Sheet 로드 중 오류 발생: {e}")
        df_bm = None
else:
    print("\n⚠️ Google 인증에 실패하여 Google Sheet 로드 작업이 중단되었습니다.")


📄 Google Sheet 'BM' 로드 중...

📄 Google Sheet 'BM' 미리보기:


,No,서비스(소),출동/영상,L형/i형,★실적집계용(출동/영상),저장장치분류,저장장치분류,21년 분류,22년 분류
0,1,금융장애처리,출동보안,L형,금융출동,,,금융출동,금융출동
1,2,GiGAeyes Basic,출동보안,L형,출동Only,,,G-B,Basic
2,3,GiGAeyes Bell,출동보안,L형,출동Only,,,G-B,Basic
3,4,Alarm,출동보안,L형,출동+영상,DVR,그 외(SD카드 등),G-G-L(DVR),Guard(DVR)
4,5,Alarm(무선)-Home,출동보안,L형,출동Only,,,G-B,Basic


❌ Google Sheet 로드 중 오류 발생: name 'sheet_bm' is not defined


**Reasoning**:
There was a `NameError` because `sheet_bm` was used instead of `sheet_name_bm` in the print statement. Correct the variable name and display the columns of `df_bm`.



In [ ]:
sheet_name_bm = 'BM'
df_bm = None

if gc:
    try:
        sh = gc.open_by_url(google_sheet_url)
        try:
            worksheet_bm = sh.worksheet(sheet_name_bm)
            print(f"\n📄 Google Sheet '{sheet_name_bm}' 로드 중...")
        except gspread.WorksheetNotFound:
            print(f"❌ 시트 '{sheet_name_bm}'를 찾을 수 없습니다.")
            print("✅ 사용 가능한 시트 목록:", [ws.title for ws in sh.worksheets()])
            worksheet_bm = None

        if worksheet_bm:
            data_bm = worksheet_bm.get_all_values()
            if data_bm:
                df_bm = pd.DataFrame(data_bm[1:], columns=data_bm[0])
                print(f"\n📄 Google Sheet '{sheet_name_bm}' 미리보기:")
                display(df_bm.head())
                print(f"\n📄 Google Sheet '{sheet_name_bm}' 컬럼:")
                display(df_bm.columns)
            else:
                print(f"❌ '{sheet_name_bm}' 시트에 데이터가 없습니다.")
                df_bm = None

    except gspread.SpreadsheetNotFound:
        print(f"❌ 해당 URL에서 Google Sheet를 찾을 수 없습니다: {google_sheet_url}")
        df_bm = None
    except Exception as e:
        print(f"❌ Google Sheet 로드 중 오류 발생: {e}")
        df_bm = None
else:
    print("\n⚠️ Google 인증에 실패하여 Google Sheet 로드 작업이 중단되었습니다.")


📄 Google Sheet 'BM' 로드 중...

📄 Google Sheet 'BM' 미리보기:


,No,서비스(소),출동/영상,L형/i형,★실적집계용(출동/영상),저장장치분류,저장장치분류,21년 분류,22년 분류
0,1,금융장애처리,출동보안,L형,금융출동,,,금융출동,금융출동
1,2,GiGAeyes Basic,출동보안,L형,출동Only,,,G-B,Basic
2,3,GiGAeyes Bell,출동보안,L형,출동Only,,,G-B,Basic
3,4,Alarm,출동보안,L형,출동+영상,DVR,그 외(SD카드 등),G-G-L(DVR),Guard(DVR)
4,5,Alarm(무선)-Home,출동보안,L형,출동Only,,,G-B,Basic



📄 Google Sheet 'BM' 컬럼:


Index(['No', '서비스(소)', '출동/영상', 'L형/i형', '★실적집계용(출동/영상)', '저장장치분류', '저장장치분류',
       '21년 분류', '22년 분류'],
      dtype='object')

## 데이터 병합

### Subtask:
BM 시트와 `df_sheet_updated` DataFrame을 '서비스(소)' 컬럼을 기준으로 병합하여 '출동/영상'과 'L형/i형' 컬럼을 추가합니다.


**Reasoning**:
Merge df_sheet_updated and df_bm on '서비스(소)' column, selecting only necessary columns from df_bm.



In [ ]:
# Select only necessary columns from df_bm
df_bm_subset = df_bm[['서비스(소)', '출동/영상', 'L형/i형']]

# Perform left merge on '서비스(소)'
df_merged_bm = pd.merge(df_sheet_updated, df_bm_subset, on='서비스(소)', how='left')

# Display the head and columns of the merged DataFrame
print("\n✅ Merged DataFrame with '출동/영상' and 'L형/i형':")
display(df_merged_bm.head())
print("\n📌 Merged DataFrame 컬럼:")
display(df_merged_bm.columns)


✅ Merged DataFrame with '출동/영상' and 'L형/i형':


,실적채널,영업구역정보,서비스(소),월정료(VAT미포함),설치주소,사업용구분,순번,계약번호,청구번호,고객번호,...,우편번호,주소,최초서비스개시일,서비스개시일시,서비스종료일시,관리본부,관리지사,계약일자,출동/영상,L형/i형
0,SP,J000605,GiGAeyes Basic,55000.0,서울 구로구 구로동 235 한신아이티타워 1214호,비사업용,1,51394537,41508200,30679589,...,152-768,서울 구로구 구로동 235 한신아이티타워 1214호,2021-05-07,2024-10-02 0:00,2025-10-01 23:59,서부본부,관악지사,2021-05-03,출동보안,L형
1,SP,M000305,GiGAeyes i-slim,17000.0,경북 안동시 와룡면 산야리 산야리 1266번지,비사업용,2,51396853,41510661,30680448,...,760-915,경북 안동시 와룡면 산야리 산야리 1266번지,2021-05-07,2025-05-07 0:00,2026-05-06 23:59,대구/경북본부,구미지사,2023-12-18,영상보안,i형
2,SP,M000305,GiGAeyes i-slim,17000.0,경북 영주시 영주동 365-43번지 와플대학 영주캠퍼스,비사업용,3,51400484,41514513,30681771,...,750-010,경북 영주시 영주동 365-43번지 와플대학 영주캠퍼스,2021-05-12,2025-05-12 0:00,2026-05-11 23:59,대구/경북본부,구미지사,2021-05-10,영상보안,i형
3,SP,O000702,GiGAeyes i-slim,27000.0,전남 목포시 상동 893-2번지 2층,비사업용,4,51401196,41712326,30784303,...,530-826,전남 목포시 상동 893-2번지 2층,2021-05-12,2025-05-12 0:00,2026-05-11 23:59,전남/전북본부,목포지사,2021-05-11,영상보안,i형
4,SP,M000305,GiGAeyes i-slim,47000.0,경북 영양군 영양읍 상원리 상원리 302-1번지,비사업용,5,51402415,41516527,30683482,...,764-803,경북 영양군 영양읍 상원리 상원리 302-1번지,2021-05-12,2025-05-12 0:00,2026-05-11 23:59,대구/경북본부,구미지사,2021-05-12,영상보안,i형



📌 Merged DataFrame 컬럼:


Index(['실적채널', '영업구역정보', '서비스(소)', '월정료(VAT미포함)', '설치주소', '사업용구분', '순번',
       '계약번호', '청구번호', '고객번호', '조회구분', '(정지/설변)기간', '이벤트시작일', '당해년도 이벤트횟수',
       '고객명', '상호', '법인명', '대표자명', '활동내역', '활동내역등록일', '활동내역등록자', '부실가입자 사유',
       '정지시작일', '정지만료일', '외국인여부', '주민등록번호', '사업자등록번호', '법인번호', '계약상태_대',
       '계약상태_중', '우편번호', '주소', '최초서비스개시일', '서비스개시일시', '서비스종료일시', '관리본부',
       '관리지사', '계약일자', '출동/영상', 'L형/i형'],
      dtype='object')

## Google sheet 업데이트

### Subtask:
병합된 데이터를 '부실0915_업데이트' 시트에 다시 씁니다.


**Reasoning**:
Write the merged and updated DataFrame to the Google Sheet, handling potential errors during the process.



In [ ]:
import pandas as pd
import gspread
from google.colab import auth
from google.auth import default

# === 1. 구글 인증 ===
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

# === 2. 구글 시트 열기 ===
sheet_url = "https://docs.google.com/spreadsheets/d/1IGWbBlPYqS7mvv9gfFISyvg6vLeJSAHXfyewt_W5pqE/edit?usp=sharing"
spreadsheet = gc.open_by_url(sheet_url)
ws_update = spreadsheet.worksheet("부실0915_업데이트")

# === 3. 데이터 불러오기 ===
df_update = pd.DataFrame(ws_update.get_all_records())

# === 4. 컬럼명 변경 ===
df_update = df_update.rename(columns={
    "관리본부명": "본부",
    "관리지사명": "지사"
})

# === 5. 본부 값 정리 ===
df_update["본부"] = df_update["본부"].str.replace("본부", "", regex=False)  # "본부" 글자 제거
df_update["본부"] = df_update["본부"].replace({
    "강원": "강북/강원",
    "서부": "강남/서부"
})

# === 6. 지사 값 정리 ===
df_update["지사"] = df_update["지사"].str.replace("지사", "", regex=False)  # "지사" 글자 제거
df_update["지사"] = df_update["지사"].replace({
    "춘천고객지원팀": "원주"
})

# === 7. 시트에 반영 (전체 업데이트) ===
ws_update.clear()
ws_update.update([df_update.columns.tolist()] + df_update.values.tolist())

print("✅ 부실0915_업데이트 시트 컬럼 및 값 변경 완료")

In [ ]:
### 아래 사용

In [ ]:
import pandas as pd
import gspread
from google.colab import auth
from google.auth import default

# ========== 0) 보조 함수 ==========
def norm_bonbu(s: str) -> str:
    s = str(s).replace("\u00A0", " ").strip()   # NBSP 제거
    s = s.replace("본부", "").strip()
    if s == "강원": s = "강북/강원"
    if s == "서부": s = "강남/서부"
    return s

def norm_jisa(s: str) -> str:
    s = str(s).replace("\u00A0", " ").strip()
    s = s.replace("지사", "").strip()
    if s == "춘천고객지원팀": s = "원주"
    return s

def make_numeric_row(values):
    return [int(v) if pd.notna(v) else 0 for v in values]

# ========== 1) 인증 및 워크북 열기 ==========
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

sheet_url = "https://docs.google.com/spreadsheets/d/1IGWbBlPYqS7mvv9gfFISyvg6vLeJSAHXfyewt_W5pqE/edit?usp=sharing"
spreadsheet = gc.open_by_url(sheet_url)

ws_bsil = spreadsheet.worksheet("부실0915_업데이트")
ws_jipo = spreadsheet.worksheet("지표")
ws_real = spreadsheet.worksheet("실제지표피벗")

# ========== 2) 원본 데이터 불러오기 ==========
df_bsil = pd.DataFrame(ws_bsil.get_all_records())
df_bsil.columns = df_bsil.columns.str.strip()

use_cols = ["계약번호", "본부", "지사", "L형/i형", "월정료(VAT미포함)"]
df = df_bsil[use_cols].copy()

# 정규화
df["본부"] = df["본부"].apply(norm_bonbu)
df["지사"] = df["지사"].apply(norm_jisa)
df["월정료"] = pd.to_numeric(df["월정료(VAT미포함)"], errors="coerce").fillna(0)

# ========== 3) 집계 → Pivot ==========
agg = df.groupby(["본부","지사","L형/i형"]).agg(
    건수=("계약번호", "count"),
    월정료=("월정료", "sum")
).reset_index()

pivot = agg.pivot_table(
    index=["본부","지사"], columns="L형/i형", values=["건수","월정료"], fill_value=0
)
pivot.columns = [f"{val}_{col}" for val, col in pivot.columns]
pivot = pivot.reset_index()

pivot["건수_총합계"] = pivot["건수_L형"] + pivot["건수_i형"]
pivot["월정료_총합계"] = pivot["월정료_L형"] + pivot["월정료_i형"]

# 월정료 천원 단위
for c in ["월정료_L형","월정료_i형","월정료_총합계"]:
    pivot[c] = (pivot[c] / 1000).astype(int)

# ========== 4) 정렬/풀리스트(빈 지사 강제 포함) ==========
bonbu_order = ["강남/서부", "강북/강원", "부산/경남", "전남/전북", "충남/충북", "대구/경북"]
jisa_order = {
    "강남/서부": ["강남","수원","분당","강동","용인","평택","인천","강서","부천","안산","안양","관악"],
    "강북/강원": ["중앙","강북","서대문","고양","의정부","남양주","강릉","원주"],
    "부산/경남": ["동부산","남부산","창원","서부산","김해","울산","진주"],
    "전남/전북": ["광주","전주","익산","북광주","순천","제주","목포"],
    "충남/충북": ["서대전","충북","천안","대전","충남서부"],
    "대구/경북": ["동대구","서대구","구미","포항"]
}

full_df = pd.DataFrame([(b, j) for b, jl in jisa_order.items() for j in jl], columns=["본부","지사"])
pivot_full = full_df.merge(pivot, on=["본부","지사"], how="left").fillna(0)

for c in ["건수_L형","건수_i형","건수_총합계","월정료_L형","월정료_i형","월정료_총합계"]:
    pivot_full[c] = pivot_full[c].astype(int)

pivot_full["본부"] = pd.Categorical(pivot_full["본부"], categories=bonbu_order, ordered=True)
pivot_full["지사순서"] = pivot_full.apply(lambda r: jisa_order[r["본부"]].index(r["지사"]) if r["지사"] in jisa_order[r["본부"]] else 999, axis=1)
pivot_sorted = pivot_full.sort_values(["본부","지사순서"]).drop(columns=["지사순서"])

# 소계 추가
rows = []
for b in bonbu_order:
    sub = pivot_sorted[pivot_sorted["본부"] == b]
    if sub.empty: continue
    rows.append(sub)
    st = sub.sum(numeric_only=True)
    st["본부"], st["지사"] = b, "소계"
    rows.append(pd.DataFrame([st]))
pivot_final = pd.concat(rows, ignore_index=True)

# 전체합계 (소계 포함 X)
grand = pivot_sorted.sum(numeric_only=True)
grand["본부"], grand["지사"] = "합계", ""
pivot_final_with_total = pd.concat([pivot_final, pd.DataFrame([grand])], ignore_index=True)

# ========== 5) 실제지표피벗 시트 갱신 ==========
cols_out = ["본부","지사","건수_L형","건수_i형","건수_총합계","월정료_L형","월정료_i형","월정료_총합계"]
view = pivot_final_with_total[cols_out].copy()
for c in cols_out[2:]:
    view[c] = view[c].astype(int).map(lambda x: f"{x:,}")
ws_real.clear()
ws_real.update([view.columns.tolist()] + view.values.tolist())
print("✅ 실제지표피벗 시트 업데이트 완료")

# ========== 6) 지표 시트 I~N 열 업데이트 ==========
df_jipo = pd.DataFrame(ws_jipo.get_all_values())
updates = []

# 본부 합계 (I5:N10)
for r in range(5, 11):
    b = norm_bonbu(df_jipo.iloc[r-1, 0])
    vals = [
        int(pivot_full[pivot_full["본부"]==b]["건수_L형"].sum()),
        int(pivot_full[pivot_full["본부"]==b]["건수_i형"].sum()),
        int(pivot_full[pivot_full["본부"]==b]["건수_총합계"].sum()),
        int(pivot_full[pivot_full["본부"]==b]["월정료_L형"].sum()),
        int(pivot_full[pivot_full["본부"]==b]["월정료_i형"].sum()),
        int(pivot_full[pivot_full["본부"]==b]["월정료_총합계"].sum())
    ]
    updates.append({"range": f"I{r}:N{r}", "values": [make_numeric_row(vals)]})

# 지사/소계 (I17:N65)
for r in range(17, 66):
    j = norm_jisa(df_jipo.iloc[r-1, 1])
    b = norm_bonbu(df_jipo.iloc[r-1, 0])
    if j == "소계":
        sub = pivot_full[pivot_full["본부"]==b]
    else:
        sub = pivot_full[(pivot_full["본부"]==b) & (pivot_full["지사"]==j)]
    if sub.empty:
        vals = [0,0,0,0,0,0]
    else:
        vals = [
            int(sub["건수_L형"].sum()),
            int(sub["건수_i형"].sum()),
            int(sub["건수_총합계"].sum()),
            int(sub["월정료_L형"].sum()),
            int(sub["월정료_i형"].sum()),
            int(sub["월정료_총합계"].sum())
        ]
    updates.append({"range": f"I{r}:N{r}", "values": [make_numeric_row(vals)]})

# 전체 합계 (I66)
r = 66
vals = [
    int(pivot_sorted["건수_L형"].sum()),
    int(pivot_sorted["건수_i형"].sum()),
    int(pivot_sorted["건수_총합계"].sum()),
    int(pivot_sorted["월정료_L형"].sum()),
    int(pivot_sorted["월정료_i형"].sum()),
    int(pivot_sorted["월정료_총합계"].sum())
]
updates.append({"range": f"I{r}:N{r}", "values": [make_numeric_row(vals)]})

ws_jipo.batch_update(updates)
print("✅ 지표 시트 I~N 값 업데이트 완료")

# ========== 7) 색상 강조 ==========
format_requests = []

# 본부 합계 5~10행 (연회색)
for r in range(5, 11):
    format_requests.append({
        "repeatCell": {
            "range": {"sheetId": ws_jipo.id, "startRowIndex": r-1, "endRowIndex": r, "startColumnIndex": 0, "endColumnIndex": 14},
            "cell": {"userEnteredFormat": {"backgroundColor": {"red": 0.95, "green": 0.95, "blue": 0.95}}},
            "fields": "userEnteredFormat.backgroundColor"
        }
    })

# 소계 17~65행
for r in range(17, 66):
    if norm_jisa(df_jipo.iloc[r-1, 1]) == "소계":
        format_requests.append({
            "repeatCell": {
                "range": {"sheetId": ws_jipo.id, "startRowIndex": r-1, "endRowIndex": r, "startColumnIndex": 0, "endColumnIndex": 14},
                "cell": {"userEnteredFormat": {"backgroundColor": {"red": 0.9, "green": 0.9, "blue": 0.9}}},
                "fields": "userEnteredFormat.backgroundColor"
            }
        })

# 합계 66행 (연노랑)
format_requests.append({
    "repeatCell": {
        "range": {"sheetId": ws_jipo.id, "startRowIndex": 65, "endRowIndex": 66, "startColumnIndex": 0, "endColumnIndex": 14},
        "cell": {"userEnteredFormat": {"backgroundColor": {"red": 1, "green": 1, "blue": 0.8}}},
        "fields": "userEnteredFormat.backgroundColor"
    }
})

if format_requests:
    spreadsheet.batch_update({"requests": format_requests})
    print("✅ 색상 적용 완료 (본부합계=연회색, 소계=회색, 합계=연노랑)")

✅ 실제지표피벗 시트 업데이트 완료
✅ 지표 시트 I~N 값 업데이트 완료
✅ 색상 적용 완료 (본부합계=연회색, 소계=회색, 합계=연노랑)


In [ ]:
import pandas as pd
import gspread
from google.colab import auth
from google.auth import default

# ===== 1) 인증 및 시트 열기 =====
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

sheet_url = "https://docs.google.com/spreadsheets/d/1IGWbBlPYqS7mvv9gfFISyvg6vLeJSAHXfyewt_W5pqE/edit?usp=sharing"
spreadsheet = gc.open_by_url(sheet_url)

ws_jipo = spreadsheet.worksheet("지표")
ws_real = spreadsheet.worksheet("실제지표피벗")

# ===== 2) 데이터 불러오기 =====
# 실제지표피벗
df_real = pd.DataFrame(ws_real.get_all_values())
df_real.columns = df_real.iloc[0]   # 첫 행을 컬럼으로
df_real = df_real.drop(0).reset_index(drop=True)

df_real["본부"] = df_real["본부"].astype(str).str.strip()
df_real["지사"] = df_real["지사"].astype(str).str.strip()

# 지표
df_jipo = pd.DataFrame(ws_jipo.get_all_values())

# ===== 3) I~N 업데이트 준비 =====
updates = []

for r in range(5, 67):   # 5행~66행
    bonbu = str(df_jipo.iloc[r-1, 0]).strip()
    jisa = str(df_jipo.iloc[r-1, 1]).strip()

    # 실제지표피벗에서 검색
    sub = df_real[(df_real["본부"]==bonbu) & (df_real["지사"]==jisa)]
    if sub.empty:
        # 소계/합계 처리
        if jisa == "소계":
            sub = df_real[(df_real["본부"]==bonbu) & (df_real["지사"]=="소계")]
        elif bonbu == "합계":
            sub = df_real[df_real["본부"]=="합계"]

    if not sub.empty:
        vals = sub.iloc[0, 2:].tolist()  # 건수/월정료 6개 값
    else:
        vals = ["0","0","0","0","0","0"]

    updates.append({"range": f"I{r}:N{r}", "values": [vals]})

# ===== 4) Batch Update 실행 =====
ws_jipo.batch_update(updates)
print("✅ 지표 시트 I~N 값 업데이트 완료 (실제지표피벗 값 그대로 반영, 천단위 콤마 유지)")

# ===== 5) 색상 강조 =====
format_requests = []

# 본부 합계 (5~10행 → 연회색)
for r in range(5, 11):
    format_requests.append({
        "repeatCell": {
            "range": {
                "sheetId": ws_jipo.id,
                "startRowIndex": r-1, "endRowIndex": r,
                "startColumnIndex": 0, "endColumnIndex": 14
            },
            "cell": {"userEnteredFormat": {"backgroundColor": {"red": 0.95, "green": 0.95, "blue": 0.95}}},
            "fields": "userEnteredFormat.backgroundColor"
        }
    })

# 소계 (17~65행, B열이 '소계'인 경우 → 회색)
for r in range(17, 66):
    jisa = str(df_jipo.iloc[r-1, 1]).strip()
    if jisa == "소계":
        format_requests.append({
            "repeatCell": {
                "range": {
                    "sheetId": ws_jipo.id,
                    "startRowIndex": r-1, "endRowIndex": r,
                    "startColumnIndex": 0, "endColumnIndex": 14
                },
                "cell": {"userEnteredFormat": {"backgroundColor": {"red": 0.9, "green": 0.9, "blue": 0.9}}},
                "fields": "userEnteredFormat.backgroundColor"
            }
        })

# 합계 (66행 → 연노랑)
format_requests.append({
    "repeatCell": {
        "range": {
            "sheetId": ws_jipo.id,
            "startRowIndex": 65, "endRowIndex": 66,
            "startColumnIndex": 0, "endColumnIndex": 14
        },
        "cell": {"userEnteredFormat": {"backgroundColor": {"red": 1, "green": 1, "blue": 0.8}}},
        "fields": "userEnteredFormat.backgroundColor"
    }
})

if format_requests:
    spreadsheet.batch_update({"requests": format_requests})
    print("✅ 색상 적용 완료 (본부합계=연회색, 소계=회색, 합계=연노랑)")

✅ 지표 시트 I~N 값 업데이트 완료 (실제지표피벗 값 그대로 반영, 천단위 콤마 유지)
✅ 색상 적용 완료 (본부합계=연회색, 소계=회색, 합계=연노랑)


In [ ]:
=IF(COUNTIF($J$2:$J$7000, J2) > 1, "중복", "고유")